<a href="https://colab.research.google.com/github/jjuhyeok/Anomaly_Detection/blob/David/LSTM%2BCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import random
import time
import math
import pickle
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from tqdm import tqdm
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from easydict import EasyDict as edict
from torch.optim.optimizer import Optimizer

from google.colab import drive
from google.colab import files

import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer, KBinsDiscretizer
from sklearn.model_selection import train_test_split

### **Mount Drive**

In [ ]:
drive.mount('/content/drive')
root = '/content/drive/My Drive/smartfactory'
# os.chdir(root)

### **Helper Functions**

In [ ]:
def feature_engineering(df):
  """
      Feature engineering using actual physical laws
  """
  df["heat_rate"] = df['air_inflow'] * (df['air_end_temp'] - 25)
  df["power_consumption"] = df["motor_current"] * df["motor_vibe"]
  df["power_output"] = df["motor_current"] * df["motor_rpm"]
  df["compressor_efficiency"] = df["motor_vibe"] / df["motor_rpm"]
  df["compressor_temp_change"] = df["air_end_temp"] - 25
  df["compressor_heat_reject"] = 1.293 * df["compressor_temp_change"] * df["air_inflow"]
  df["air_mass_flow"] = 1.293 * df["air_inflow"]
  df['air_velocity'] = df["air_inflow"] / (3.14 * 0.05 * 0.05)
  df['air_pressure'] = 101.325 + 0.5 * 1.293 * (df["air_velocity"]**2)
  df["air_enthalpy"] = 700 * df["air_inflow"] * df["air_end_temp"]
  df["compression_ratio"] = df["out_pressure"] / 101.325
  df["temp_pressure_ration"] = (25 / df["air_end_temp"]) * (101.325 / df["out_pressure"])
  return df


def to_numeric(df):
  """
      Change numerics to numeric type
  """
  
  for col in list(df.columns):
      df[col] = df[col].apply(pd.to_numeric, errors='ignore')
  return df


def get_columns(df):
  return list(df.columns)


def csv_download(df, filename):
  df.to_csv(f'{filename}.csv', index=False)
  files.download(f'{filename}.csv')


def filter_data(df, idx):
  df1 = pd.DataFrame()
  for id in idx:
    df1 = pd.concat([df1, df[df["type"] == id]], axis=0)
  return df1


### **Preprocessing**

##### **Load and Feature Engineer Data**

In [ ]:
train = pd.read_csv(os.path.join(root, "train_data.csv"))
test = pd.read_csv(os.path.join(root, "test_data.csv"))

In [ ]:
train = feature_engineering(train)
test = feature_engineering(test)

columns = get_columns(train)
columns = [x for x in columns if x != "type"]

##### **Scaling**

In [ ]:
train_on_idx = [4]      # Select the class types you want to train on
df_train = filter_data(train, train_on_idx)

In [ ]:
# scaler = MinMaxScaler(clip=True)
scaler = StandardScaler()
train_data = scaler.fit_transform(df_train[columns])
train_data = pd.DataFrame(data=train_data, index=df_train.index.tolist(), columns=columns)
train_data = pd.concat([train_data, df_train[['type']]], axis=1)

In [ ]:
train_inv_transform = np.vstack((scaler.mean_, scaler.scale_))
train_inv_transform = pd.DataFrame(train_inv_transform, columns=columns, index=["mean", "stdv"])

##### **Checkpoint**

In [ ]:
trial = 0
ckpt_dir = Path(f'{root}/logs_competition/ckpt_{trial}')
if ckpt_dir.exists():
    shutil.rmtree(ckpt_dir)

ckpt_dir.mkdir(parents=True)

In [ ]:
pickle.dump(train_inv_transform, open(f'{ckpt_dir}/train_ss.pkl','wb'))

##### **Train-Val Split**

In [ ]:
def train_val_split(df, split, start=None):
  """"
      Randomly take a portion of data
  """
  val = int(len(df) * (1 - split))
  if val != 0:
    index = random.randint(0, len(df) - val)

    train = pd.concat([df.iloc[0 : index, :], df.iloc[index + val: , :]], axis=0)
    valid = df.iloc[index: index + val, :]
    if start:
      index = start
    else:
      index = valid.index.tolist()[0]

    return train, valid, index
  
  return df, pd.DataFrame(columns=list(df.columns)), None


def class_based_split(df, split, idx, start):
  """
      Keep the split proportion within each class
  """
  train = pd.DataFrame()
  val = pd.DataFrame()
  start_ = []
  for i, id in enumerate(idx):
    type_i = df[df['type'] == id]
    type_i_train, type_i_val, type_i_index = train_val_split(type_i, split, start[i])
    start_.append(type_i_index)
    train = pd.concat([train, type_i_train], axis=0, ignore_index=True)
    val = pd.concat([val, type_i_val], axis=0, ignore_index=True)
  return train, val, start_


In [ ]:
start = [None]*len(train_on_idx)
training_data, validation_data, start = class_based_split(train_data, split=0.8, idx=train_on_idx, start=start)
print(f"Validation start index for each type {train_on_idx} is {start}")

##### **Construct Dataset**

In [ ]:
def get_prev_data(df, index, backward, features, inclusive=True):
    start = index - backward
    end = index
    if start < 0:
        df_temp = df.loc[0 : end, features]
        df_temp = pd.concat([pd.DataFrame.from_records([[0.]*len(features)]*np.abs(start), columns=features), df_temp], axis=0)
        df_temp = df_temp.replace(to_replace=0.0, method='bfill')

    else:
        df_temp = df.loc[start : end, features]

    if not inclusive:
        df_temp = df_temp.iloc[:-1]

    df_temp.reset_index(drop=True, inplace=True)
    return df_temp

In [ ]:
class MyDataset(Dataset):
  def __init__(self, data, features, added_features, look_forward, look_backward, duplicate=0):
    """
        data: Input data
        features: Features reconstruction is done on
        added_features: Feature engineered or other features
        look_forward: a list of numbers that will be randomly selected. Determines our time window to the future
        look_backwaed: a list of numbers that will be randomly selected. Determines our time window to the past
        duplicat: how many times do you want to duplicate each sequence
    """
    self.input = []
    self.prev = []
    self.additional = []
    size = len(data)

    for i in tqdm(range(size)):
      forward = random.choice(look_forward)
      backward = random.choice(look_backward)
      
      prev_data = get_prev_data(data, i, backward, features + added_features, inclusive=True)
      input_data = data.loc[[i], features]
      additional_data = data.loc[[i], added_features]

      if duplicate:
        for i in range(duplicate):
          self.prev.append(prev_data.values.tolist())
          self.input.append(input_data.iloc[0, :].values.tolist())
          self.additional.append(additional_data.iloc[0, :].values.tolist())

      self.prev.append(prev_data.values.tolist())
      self.input.append(input_data.iloc[0, :].values.tolist())
      self.additional.append(additional_data.iloc[0, :].values.tolist())

    self.input = torch.FloatTensor(self.input)            
    self.prev = torch.FloatTensor(self.prev)        
    self.additional = torch.FloatTensor(self.additional)
  
  def __getitem__(self, index):
    return {"input": self.input[index],
            "prev": self.prev[index],
            "additional": self.additional[index]
    }
  def __len__(self):
    return len(self.input)

In [ ]:
features = ['air_inflow', 'air_end_temp', 'out_pressure', 'motor_current', 'motor_rpm', 'motor_temp', 'motor_vibe']
added_features = ['heat_rate', 'power_consumption', 'power_output', 'compressor_efficiency', 'compressor_temp_change', 'compressor_heat_reject', 'air_mass_flow', 'air_velocity', 'air_pressure', 'air_enthalpy', 'compression_ratio', 'temp_pressure_ration']

train_dataset = MyDataset(data=training_data, features=features, added_features=added_features, look_forward=[0], look_backward=[2], duplicate=10)
val_dataset = MyDataset(data=validation_data, features=features, added_features=added_features, look_forward=[0], look_backward=[2], duplicate=0)

In [ ]:
batch_size = 64
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)

### **Training**

In [ ]:
# code inspired from: https://github.com/anandsaha/pytorch.cyclic.learning.rate/blob/master/cls.py
class CyclicLR(object):
    def __init__(self, optimizer, base_lr=1e-3, max_lr=6e-3,
                 step_size=2000, mode='triangular', gamma=1.,
                 scale_fn=None, scale_mode='cycle', last_batch_iteration=-1):

        if not isinstance(optimizer, Optimizer):
            raise TypeError('{} is not an Optimizer'.format(
                type(optimizer).__name__))
        self.optimizer = optimizer

        if isinstance(base_lr, list) or isinstance(base_lr, tuple):
            if len(base_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} base_lr, got {}".format(
                    len(optimizer.param_groups), len(base_lr)))
            self.base_lrs = list(base_lr)
        else:
            self.base_lrs = [base_lr] * len(optimizer.param_groups)

        if isinstance(max_lr, list) or isinstance(max_lr, tuple):
            if len(max_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} max_lr, got {}".format(
                    len(optimizer.param_groups), len(max_lr)))
            self.max_lrs = list(max_lr)
        else:
            self.max_lrs = [max_lr] * len(optimizer.param_groups)

        self.step_size = step_size

        if mode not in ['triangular', 'triangular2', 'exp_range'] \
                and scale_fn is None:
            raise ValueError('mode is invalid and scale_fn is None')

        self.mode = mode
        self.gamma = gamma

        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = self._triangular_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = self._triangular2_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = self._exp_range_scale_fn
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode

        self.batch_step(last_batch_iteration + 1)
        self.last_batch_iteration = last_batch_iteration

    def batch_step(self, batch_iteration=None):
        if batch_iteration is None:
            batch_iteration = self.last_batch_iteration + 1
        self.last_batch_iteration = batch_iteration
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

    def _triangular_scale_fn(self, x):
        return 1.

    def _triangular2_scale_fn(self, x):
        return 1 / (2. ** (x - 1))

    def _exp_range_scale_fn(self, x):
        return self.gamma**(x)

    def get_lr(self):
        step_size = float(self.step_size)
        cycle = np.floor(1 + self.last_batch_iteration / (2 * step_size))
        x = np.abs(self.last_batch_iteration / step_size - 2 * cycle + 1)

        lrs = []
        param_lrs = zip(self.optimizer.param_groups, self.base_lrs, self.max_lrs)
        for param_group, base_lr, max_lr in param_lrs:
            base_height = (max_lr - base_lr) * np.maximum(0, (1 - x))
            if self.scale_mode == 'cycle':
                lr = base_lr + base_height * self.scale_fn(cycle)
            else:
                lr = base_lr + base_height * self.scale_fn(self.last_batch_iteration)
            lrs.append(lr)
        return lrs

##### **Architecture**

In [ ]:
class MyLSTM(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim_1, hidden_dim_2, hidden_dim_3, hidden_dim_4, lstm_layer=2):
        super(MyLSTM, self).__init__()
        
        self.embedding = nn.Sequential(
            nn.Conv1d(in_channels=input_dim, out_channels=hidden_dim_1,  kernel_size=3, padding=1, stride=1, bias=False),
            nn.ReLU()
        )  


        self.lstm1 = nn.LSTM(input_size=hidden_dim_1, hidden_size=hidden_dim_2, num_layers=1, bidirectional=True)
        self.conv_layers1 = nn.Sequential(
            nn.Conv1d(in_channels=hidden_dim_2*2, out_channels=hidden_dim_1,  kernel_size=3, padding=1, stride=1, bias=False),
            nn.ReLU()
        )        


        self.lstm2 = nn.LSTM(input_size=hidden_dim_2*2, hidden_size=hidden_dim_2, num_layers=1, bidirectional=True)
        self.conv_layers2 = nn.Sequential(
            nn.Conv1d(in_channels=hidden_dim_2*2, out_channels=hidden_dim_1,  kernel_size=5, padding=2, stride=1, bias=False),
            nn.ReLU()
        )

        self.dropout = nn.Dropout(p=0.2)
        
        
        self.fc1 = nn.Sequential(
                                    nn.Linear(in_features=hidden_dim_1*4 + output_dim, out_features=hidden_dim_2),
                                    nn.BatchNorm1d(hidden_dim_2),
                                    nn.ReLU(),
                                    nn.Dropout(0.2)
                                )

        self.fc2 = nn.Sequential(
                            nn.Linear(in_features=hidden_dim_2, out_features=hidden_dim_3),
                            nn.BatchNorm1d(hidden_dim_3),
                            nn.ReLU(),
                            nn.Dropout(0.2)
                        )
                
        self.fc3 = nn.Sequential(
                            nn.Linear(in_features=hidden_dim_3, out_features=hidden_dim_1),
                            nn.BatchNorm1d(hidden_dim_1),
                            nn.ReLU()
                        ) 

        self.fc4 = nn.Linear(in_features=hidden_dim_1, out_features=output_dim)


    def forward(self, x, y, z):
        """
            x: Input data of past coils
            y: Current info
            z: Current added info
        """
        # x: [batch_size, seq_len, embedding]
        # Permute tensor to (batch_size, embedding_dim, seq_len)
        x = x.permute(0, 2, 1)                      # (B, emb, seq_len)
        x = self.embedding(x)
        
        x = x.permute(0, 2, 1)                      # (B, seq_len, emb)
        out1, (h_n, c_n) = self.lstm1(x)            # (B, seq_len, hidden_dim_2*2)
        
        x = out1.permute(0, 2, 1)
        x_1 = self.conv_layers1(x)
        x_1 = nn.functional.max_pool1d(x_1, kernel_size=x_1.size()[2:])

        x_2 = self.conv_layers2(x)
        x_2 = nn.functional.max_pool1d(x_2, kernel_size=x_2.size()[2:])


        out2, (h_n, c_n) = self.lstm2(out1)
        
        x = out2.permute(0, 2, 1)
        y_1 = self.conv_layers1(x)
        y_1 = nn.functional.max_pool1d(y_1, kernel_size=y_1.size()[2:])

        y_2 = self.conv_layers2(x)
        y_2 = nn.functional.max_pool1d(y_2, kernel_size=y_2.size()[2:])


        x = torch.cat([x_1, x_2, y_1, y_2], dim=1).squeeze(2)
        x = torch.cat([x, y], dim=1)
        
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        return x

In [ ]:
num_epoch = 600

##### **Trainer**

In [ ]:
def train(model, optimizer, criterion, scheduler):
  best_loss = 9999999
  training_mae_loss, training_mse_loss = [], []
  validation_mae_loss, validation_mse_loss = [], []
  ep = 0
    
  if True in [x.endswith(".pt") for x in os.listdir(ckpt_dir)]:
    try:
      checkpoint = torch.load(f'{ckpt_dir}/{trial}_best.pt')
      model.load_state_dict(checkpoint['model_state_dict'])
      optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
      ep = checkpoint['epoch'] + 1
      best_loss = checkpoint['best_loss']
      print(f"Resuming from previous episode: {ep}, best_loss: {best_loss}")
    except:
      print(f"Something wrong with {ckpt_dir}/{trial}_best.pt model. Please remove it!")


  for epoch in range(ep, num_epoch):
    # Here starts the train loop.
    model.train()

    N_total = 0.
    MAE_total = 0.
    MSE_total = 0.
    R_SQUARE_total = 0.

    for batch_idx, data in tqdm(enumerate(train_dataloader)):
      input = data['input'].cuda()
      prev = data['prev'].cuda()
      additional = data['additional'].cuda()
      
      y_pred = model(prev, input, additional)

      loss = criterion(y_pred, input)
      optimizer.zero_grad()
      loss.backward()

      optimizer.step()
      scheduler.batch_step()

      # Update the metrics for this batch
      batch_mae = torch.mean(torch.abs(y_pred - input), dim=0)
      batch_mse = torch.mean((y_pred - input)**2, dim=0)

      y_mean = torch.mean(input, dim=0, keepdim=True)
      y_diff = input - y_pred

      batch_r_squared = 1 - torch.sum(y_diff**2, dim=0) / torch.sum((input - y_mean)**2, dim=0)

      # Accumulate the metrics for this epoch
      N_total += len(data)
      MAE_total += torch.sum(batch_mae).item()
      MSE_total += torch.sum(batch_mse).item()
      

    # L_total /= N_total
    MAE_total /= N_total
    MSE_total /= N_total

    training_mae_loss.append(MAE_total)
    training_mse_loss.append(MSE_total)

    print(f'Training result {epoch}/{num_epoch} || MSE loss: {MSE_total:.5f} || MAE loss: {MAE_total:.5f}')
    
    # Here starts the validation loop.
    model.eval()
    with torch.no_grad():
      N_total_ = 0.
      MAE_total_ = 0.
      MSE_total_ = 0.
      R_SQUARE_total_ = 0.

      for batch_idx, data in tqdm(enumerate(val_dataloader)):
        input = data['input'].cuda()
        prev = data['prev'].cuda()
        additional = data['additional'].cuda()
        
        y_pred = model(prev, input, additional)
        
        loss = criterion(y_pred, input)

        # Update the metrics for this batch
        batch_mae = torch.mean(torch.abs(y_pred - input), dim=0)
        batch_mse = torch.mean((y_pred - input)**2, dim=0)

        y_mean = torch.mean(input, dim=0, keepdim=True)
        y_diff = input - y_pred
        
        batch_r_squared = 1 - torch.sum(y_diff**2, dim=0) / torch.sum((input - y_mean)**2, dim=0)

        # Accumulate the metrics for this epoch
        N_total_ += len(data)
        MAE_total_ += torch.sum(batch_mae).item()
        MSE_total_ += torch.sum(batch_mse).item()

      #L_total_ /= N_total_
      MAE_total_ /= N_total_
      MSE_total_ /= N_total_

      validation_mae_loss.append(MAE_total_)
      validation_mse_loss.append(MSE_total_)

      print(f'\tValidation result {epoch}/{num_epoch} || MSE loss: {MSE_total_:.5f} || MAE loss: {MAE_total_:.5f}')
      
      # Whenever `test_accuracy` is greater than `best_accuracy`, save network weights with the filename 'best.pt' in the directory specified by `ckpt_dir`.
      if MAE_total_ < best_loss:
        best_loss = MAE_total_
        torch.save(
            {
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'best_loss': best_loss
            },
            f'{ckpt_dir}/{trial}_best.pt')
  
  torch.save(
              {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_loss': best_loss
              },
              f'{ckpt_dir}/{trial}_all.pt')
  return training_mae_loss, training_mse_loss, validation_mae_loss, validation_mse_loss

In [ ]:
input_dim = train_dataset[0]["prev"].size()[1]
output_dim = train_dataset[0]["input"].size()[0]
hidden_dim_1 = 128
hidden_dim_2 = 64
hidden_dim_3 = 256
hidden_dim_4 = train_dataset[0]["additional"].size()[0]
lstm_layer = 2

In [ ]:
model = MyLSTM(input_dim = input_dim,
               output_dim = output_dim,
               hidden_dim_1 = hidden_dim_1,
               hidden_dim_2 = hidden_dim_2,
               hidden_dim_3 = hidden_dim_3,
               hidden_dim_4 = hidden_dim_4,
               lstm_layer = lstm_layer).cuda()

# criterion = torch.nn.MSELoss()
criterion = torch.nn.L1Loss()

base_lr, max_lr = 0.001, 0.003
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=max_lr)

step_size = 300

scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr,
               step_size=step_size, mode='exp_range',
               gamma=0.99994)

In [ ]:
train_mae_loss, train_mse_loss, val_mae_loss, val_mse_loss = train(model=model, optimizer=optimizer, criterion=criterion, scheduler=scheduler)

##### **Plot Loss**

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y= train_mae_loss,
                    mode='lines',
                    name='MAE Train Loss'))
fig.add_trace(go.Scatter(y=val_mae_loss,
                    mode='lines',
                    name='MAE Val Loss'))
fig.show()




fig = go.Figure()
fig.add_trace(go.Scatter(y= train_mse_loss,
                    mode='lines',
                    name='MSE Train Loss'))
fig.add_trace(go.Scatter(y=val_mse_loss,
                    mode='lines',
                    name='MSE Val Loss'))
fig.show()

### **Testing**

In [ ]:
criterion_1 = torch.nn.L1Loss()
criterion_2 = torch.nn.MSELoss()
criterion_3 = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

##### **Load Desired Model**

In [ ]:
new_trial = 0
model_path_ = Path(f'{root}/logs_competition/ckpt_{new_trial}/{new_trial}_all.pt')

ckpt_ = torch.load(model_path_)
print(f"Best MAE LOSS: {ckpt_['best_loss']} after training for {ckpt_['epoch']}")

In [ ]:
model_ = MyLSTM(input_dim = input_dim,
               output_dim = output_dim,
               hidden_dim_1 = hidden_dim_1,
               hidden_dim_2 = hidden_dim_2,
               hidden_dim_3 = hidden_dim_3,
               hidden_dim_4 = hidden_dim_4,
               lstm_layer = lstm_layer).cuda()

model_.load_state_dict(ckpt_["model_state_dict"])
model_.eval()

In [ ]:
test_on_idx = [4, 5]      # Select the class types you want to test on
df_test = filter_data(test, test_on_idx)

##### **Transform Test Data**

In [ ]:
test_data = scaler.transform(df_test[columns])
test_data = pd.DataFrame(data=test_data, index=df_test.index.tolist(), columns=columns)
test_data = pd.concat([test_data, df_test[['type']]], axis=1)
test_data.reset_index(drop=True, inplace=True)

In [ ]:
test_dataset = MyDataset(data=test_data, features=features, added_features=added_features, look_forward=[0], look_backward=[2], duplicate=0)

In [ ]:
test_dataloader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

In [ ]:
mae_diff = []
mse_diff = []
cosine_similarity = []
mae_original = []

for data in tqdm(test_dataloader):
        input = data['input'].cuda()
        prev = data['prev'].cuda()
        additional = data['additional'].cuda()
      
        xout = model_(prev, input, additional)

        in_val = input.squeeze(0).cpu().detach().numpy()
        in_val *= train_inv_transform.loc["stdv", features]   
        in_val += train_inv_transform.loc["mean", features]

        out_val = xout.squeeze(0).cpu().detach().numpy()
        out_val *= train_inv_transform.loc["stdv", features]   
        out_val += train_inv_transform.loc["mean", features]

        mae = criterion_1(xout, input)
        mse = criterion_2(xout, input)
        cos = criterion_3(xout, input)
        mae_orig = criterion_1(torch.FloatTensor(out_val), torch.FloatTensor(in_val))

        mae_diff.append(float(mae.cpu().detach()))
        mse_diff.append(float(mse.cpu().detach()))
        cosine_similarity.append(float(cos.cpu().detach()))
        mae_original.append(float(mae_orig.cpu().detach()))

##### **Plot Reconstruction Error**

In [ ]:
fig = px.scatter(x=df_test.index.tolist(), y=mae_diff, color=df_test["type"])
fig.update_layout(title=str(test_on_idx))
fig.show()